In [112]:
import numpy as np
import cv2 as cv

### Step 1: Read the image as a Numpy Array

In [113]:
image_path = 'meteor_challenge_01.png'
img = cv.imread(image_path)

### Step 2: Create masks and isolate white and red colors (BGR)

In [114]:
white = np.array([255, 255, 255])
red = np.array([0, 0, 255])

In [115]:
mask_star = cv.inRange(img, white, white)
mask_meteor = cv.inRange(img, red, red)

### Step 3: Using the masks, count how many stars and meteors pixels the image has

In [116]:
star_counter  = cv.countNonZero(mask_star)
meteor_counter = cv.countNonZero(mask_meteor)

In [117]:
print(f"Number of stars: {star_counter}")
print(f"Number of meteors: {meteor_counter}")

Number of stars: 315
Number of meteors: 328


### Step 4: Create a blue mask for the water

In [118]:
blue = np.array([255, 0, 0])
mask_water = cv.inRange(img, blue, blue)
water = cv.countNonZero(mask_water)

### Step 5: Iterate through each column of the image and verify if water mask and meteor mask are present

In [119]:
meteors_on_water_counter = 0
img_width = img.shape[1]

for pixel_column in range(img_width):
    if np.any(mask_water[:, pixel_column]):
        meteors_on_water_counter += cv.countNonZero(mask_meteor[:, pixel_column])

print(f"Meteors falling on water: {meteors_on_water_counter}")

Meteors falling on water: 105


### Step 6: Trying to identify the message written in the sky

First attempt: create lines connecting dots of the same color

In [120]:
# contours, _ = cv.findContours(mask_meteor, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
# for contour in contours:
#     cv.polylines(img, [contour], isClosed=False, color=(0, 0, 255), thickness=2)
# for contour in contours:
#     cv.polylines(img, [contour], isClosed=False, color=(255, 255, 255), thickness=2)
# cv.imshow("Image", img)
# cv.waitKey(0)
# cv.destroyAllWindows()

# No success


Second attempt: following hint 2, I'll check each column count of stars and meteors. Morse code? Treating stars as '-' and meteors as '.'

In [121]:
sentence = []

for pixel_column in range(img_width):
    character = ""

    if np.any(mask_star[:, pixel_column]):
        character += "."
    if np.any(mask_meteor[:, pixel_column]):
        character += "-"
    
    # if theres no dot or dash in the column, add space
    if not character:
        character = " "
    
    sentence.append(character)

hidden_message = "".join(sentence)
print(hidden_message)

 -.--  .-  .---.--.- .-.-. . - -.-- ..-. .-.-. -..  .- ---  ..- ...  .-. .-... .-.-.-.--- -.-- --- ..-    . .-.- --.- .-.--..-..- .-.-. .-.- .-.-.-.    .-      .-.--.-  - .-.- .-.-.-.- .-.-.- .-..-  .-      .-.- . -- .-.-    .- .-.-.--.-  ..-  .   -.-- -   .-.-..  .- .-.- .-..-.- .-.-. .- .-  .-      .-.- .  - .-.- .---. .-.-. .-    .- ...  -.- - -- .-- .- .- .-.-. .- - -.-- ... ..-.  .. -.- -- - .-.- ---.- .-.-- -.-  .-.- .-...- .-.-.-.--. .-.-. .---  .-      .-.- .--  .-.- .-.-.-.- .-.-.- ..-. -.-- --- .-.- .  - .-.--  -. .-.-. -.  ..-  .-   .-      ..-..  . ..- .-.-..- ..-. . . -.-  -  .-.- --.-.- .-.-   -.- .-.- .-..-- -.---  - ..-  ... .-.  .-. .-.-. . - -.- --   .-.- . -  .-.- .---.- .-.-. . -  .- ---- .-.-  -.- .-.--.-..- .-.- -.-   -.-- --  .-.- .-..- .-.-- .-.- .-.-- .- . .-.-. - - -.--  -  ..- .. . .-.-...-.- .-.-.- .-.  .-.- .  .- .-.- .-.-.  .-.- -.-.. -.- ---- .-.- -.-.-  .-.- ..-..-


In [122]:
MORSE_CODE_DICT = { 'A':'.-', 'B':'-...',
                    'C':'-.-.', 'D':'-..', 'E':'.',
                    'F':'..-.', 'G':'--.', 'H':'....',
                    'I':'..', 'J':'.---', 'K':'-.-',
                    'L':'.-..', 'M':'--', 'N':'-.',
                    'O':'---', 'P':'.--.', 'Q':'--.-',
                    'R':'.-.', 'S':'...', 'T':'-',
                    'U':'..-', 'V':'...-', 'W':'.--',
                    'X':'-..-', 'Y':'-.--', 'Z':'--..',
                    '1':'.----', '2':'..---', '3':'...--',
                    '4':'....-', '5':'.....', '6':'-....',
                    '7':'--...', '8':'---..', '9':'----.',
                    '0':'-----', ', ':'--..--', '.':'.-.-.-',
                    '?':'..--..', '/':'-..-.', '-':'-....-',
                    '(':'-.--.', ')':'-.--.-'}

# inverting key with values for comparisson
MORSE_CODE_DICT = {value: key for key, value in MORSE_CODE_DICT.items()}

# split morse symbols
morse_symbols = hidden_message.split(" ")

decoded_message = ""
for symbol in morse_symbols:
    if symbol in MORSE_CODE_DICT:
        decoded_message += MORSE_CODE_DICT[symbol]
    else:
        decoded_message += " "  # add a space for unknown symbols or gaps

print("Decoded Message:", decoded_message)

Decoded Message:  Y A   ETYF D AO US R  YOU   E Q       A       T  .  A      EM    A  U E  YT    A   AA A      E T   A   AS KTMWAA ATYSF IKMT   K     J A      W   .FYO E T  N N U A  A       EU FEEK T      K    TU SR R ETKM   ET    ET A   K  K  YM      AE TTY T UIE .R  E A     K      


Trying stars as dashes and meteors as dots

In [123]:
sentence = []

for pixel_column in range(img_width):
    character = ""

    if np.any(mask_star[:, pixel_column]):
        character += "-"
    if np.any(mask_meteor[:, pixel_column]):
        character += "."
    if not character:
        character = " "
    
    sentence.append(character)

hidden_message = "".join(sentence)
print(hidden_message)

 .-..  -.  -...-..-. -.-.- - . .-.. --.- -.-.- .--  -. ...  --. ---  -.- -.--- -.-.-.-... .-.. ... --.    - -.-. ..-. -.-..--.--. -.-.- -.-. -.-.-.-    -.      -.-..-.  . -.-. -.-.-.-. -.-.-. -.--.  -.      -.-. - .. -.-.    -. -.-.-..-.  --.  -   .-.. .   -.-.--  -. -.-. -.--.-. -.-.- -. -.  -.      -.-. -  . -.-. -...- -.-.- -.    -. ---  .-. . .. -.. -. -. -.-.- -. . .-.. --- --.-  -- .-. .. . -.-. ...-. -.-.. .-.  -.-. -.---. -.-.-.-..- -.-.- -...  -.      -.-. -..  -.-. -.-.-.-. -.-.-. --.- .-.. ... -.-. -  . -.-..  .- -.-.- .-  --.  -.   -.      --.--  - --. -.-.--. --.- - - .-.  .  -.-. ..-.-. -.-.   .-. -.-. -.--.. .-...  . --.  --- -.-  -.- -.-.- - . .-. ..   -.-. - .  -.-. -...-. -.-.- - .  -. .... -.-.  .-. -.-..-.--. -.-. .-.   .-.. ..  -.-. -.--. -.-.. -.-. -.-.. -. - -.-.- . . .-..  .  --. -- - -.-.---.-. -.-.-. -.-  -.-. -  -. -.-. -.-.-  -.-. .-.-- .-. .... -.-. .-.-.  -.-. --.--.


In [124]:
morse_symbols = hidden_message.split(" ")

# Decode each symbol into a letter
decoded_message = ""
for symbol in morse_symbols:
    if symbol in MORSE_CODE_DICT:
        decoded_message += MORSE_CODE_DICT[symbol]
    else:
        decoded_message += " "  # Add a space for unknown symbols or gaps

print("Decoded Message:", decoded_message)

Decoded Message:  L N   TELQ W NS GO K  LSG   TCF  C    N       EC  ( N     CTIC   N  G T  LE    NC  NN N     CT EC  N   NO REIDNN NELOQ MRIEC  R C   B N     CD C  QLSCT E  A A G N  N       TG QTTR E C C  RC   EG OK K TERI  CTE C  TE NHC R CR  LI C( C NT EEL E GMT  K CT NC  C RHC  C 


Since I've got no result from iterating through the columns, I'll try the rows:

In [125]:
img_height = img.shape[0]
sentence = []

for pixel_row in range(img_height):
    character = ""

    if np.any(mask_star[pixel_row, :]):  
        character += "-"
    if np.any(mask_meteor[pixel_row, :]):
        character += "."
    if not character:
        character = " "
    
    sentence.append(character)

hidden_message = "".join(sentence)
print(hidden_message)


----.----...-.--.---..-..----....--.-.---.- -..------..-.--...-.-.-----..--.---.....---..-....--.-..-.---.-.-.--.---..-..-...--.--..-.-..-...-.-..---..-...----.-...- .----- ..-.--...-. --.-- -.---- --.-- ------ -.-- --  -.--- --    - ---   -  ---   - -- --- -- - - -  -  - -  --   -   --  ----- -  - - ---- -  --- -  ---  -  -  - --- - --   ----- --  -- - --------   -- - -------     ----- -  - -- -                                                                                                                                                                                                                                                                                                                 


In [126]:
morse_symbols = hidden_message.split(" ")

# Decode each symbol into a letter
decoded_message = ""
for symbol in morse_symbols:
    if symbol in MORSE_CODE_DICT:
        decoded_message += MORSE_CODE_DICT[symbol]
    else:
        decoded_message += " "  # add a space for unknown symbols or gaps

print("Decoded Message:", decoded_message)

Decoded Message:         YM  M   TO  T O  TMOMTTT T TT M  T  M 0T TT T OT O T T TOTM  0M MT   MT     0T TMT                                                                                                                                                                                                                                                                                                                 
